# Code for evaluation

### Input:

**Name:** groundtruth  
**Type:** Dataframe  
**Column:** 'main_genres'  
**Datatype:**: List

**Name:** predictions  
**Type:** Series  
**Datatype:**: List

In [ ]:
from os import path
import pandas as pd
import numpy as np

In [ ]:
def checkIfMember(row):
    return row['p'] in row['t']

# t can be:
#    groundtruth['genre1']
#    groundtruth['main_genres']
def comparePredictions(predictions, groundtruth):
    compare = pd.DataFrame({"p":predictions, "t": groundtruth})
    compare['c'] = compare.apply(checkIfMember, axis = 1)
    
    #Test Code
#     print sum(compare['c'])
#     print compare.head()
    
    return compare


In [ ]:
def evaluatePredictions(compare):
    #Calculate totals
    length = compare['c'].shape[0]
    correct = sum(compare['c'])

    totals = pd.DataFrame(columns=['correct', 'total'])
    totals.loc['total'] = (correct, length)
    totals['accuracy'] = np.around((totals['correct'] / totals['total'] *100.00).astype(np.double),2)
    

    
    #Calculate genre results
    results = pd.DataFrame(columns=['total', 'classified', 'other', 'wrong'])

    for i, row in compare.iterrows():
        prediction = row['p']
        
        if prediction not in results.index:
                results.loc[prediction] = [0, 0, 0, 0]
        
        if prediction not in row['t']:
                results.loc[prediction]['wrong'] += 1
     
        for genre in row['t']:
            if genre not in results.index:
                results.loc[genre] = [0, 0, 0, 0]

            #Calculate total
            results.loc[genre]['total'] += 1

            #Calculate classified
            if row['c'] & (genre == row['p']):
                results.loc[genre]['classified'] += 1

            #Calculate other
            if row['c'] & (genre != row['p']):
                results.loc[genre]['other'] += 1


    results['diff'] = results['total'] - results['classified'] - results['other']
    results['classperc'] = np.around((results['classified'] / results['total'] *100.00).astype(np.double),2)
    results['otherperc'] = np.around((results['other'] / results['total'] *100.00).astype(np.double),2)
    results['wrongperc'] = np.around((100 -  results['classperc'] - results['otherperc']).astype(np.double),2)

    results = results.sort_values(ascending=False, by= 'total')
    
    display(totals)
    display(results)
    
    return (totals, results)

In [ ]:
#Do the evaluation
compare = comparePredictions(predictions, groundtruth)
results = evaluatePredictions(compare)
